# Toy RAG System with Neo4j and Llama2
https://neo4j.com/developer-blog/knowledge-graph-rag-application/


In [1]:
from langchain.graphs import Neo4jGraph

In [4]:
url = "neo4j+s://0071569e.databases.neo4j.io"
username ="neo4j"
password = "L5mjLRInl_FRXHb-AEcMh7c4CIDZU8i3lycj0mBflns"

graph = Neo4jGraph(
    url=url, 
    username=username, 
    password=password
)

In [5]:
import requests

url = "https://gist.githubusercontent.com/tomasonjo/08dc8ba0e19d592c4c3cde40dd6abcc3/raw/da8882249af3e819a80debf3160ebbb3513ee962/microservices.json"
import_query = requests.get(url).json()['query']
graph.query(
    import_query
)

[]

In [7]:
import os
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain_community.embeddings import OllamaEmbeddings

url = "neo4j+s://0071569e.databases.neo4j.io"

vector_index = Neo4jVector.from_existing_graph(
    OllamaEmbeddings(),
    url=url,
    username=username,
    password=password,
    index_name='tasks',
    node_label="Task",
    text_node_properties=['name', 'description', 'status'],
    embedding_node_property='embedding',
)

In [8]:
response = vector_index.similarity_search(
    "How will RecommendationService be updated?"
)
print(response[0].page_content)


name: Enhance
description: Enhance ShippingService
status: Completed


In [10]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOllama

vector_qa = RetrievalQA.from_chain_type(
    llm=ChatOllama(model="llama2"),
    chain_type="stuff",
    retriever=vector_index.as_retriever()
)
vector_qa.invoke(
    "How will recommendation service be updated?"
)

{'query': 'How will recommendation service be updated?',
 'result': 'Based on the context provided, it appears that the recommendation service is currently in the process of being updated with a new feature. The status of the update is listed as "In Progress." I do not have any information beyond what is provided in the context, so I cannot provide a definitive answer to how the recommendation service will be updated.'}

In [15]:
from langchain.chains import GraphCypherQAChain

graph.refresh_schema()

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm = ChatOllama(model='llama2'),
    qa_llm = ChatOllama(model='llama2'), graph=graph, verbose=True,
)

In [16]:
cypher_chain.run(
    "How many open tickets there are?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (t:Task)
WHERE t.status = 'OPEN'
RETURN COUNT(t)

Full Context:
[{'COUNT(t)': 0}]

> Finished chain.


'I apologize, but based on the information provided, I cannot provide an answer to your question as it is empty. There is no data or information provided that could help me generate a meaningful response. If you have any other questions or concerns, please feel free to ask.'